<a href="https://colab.research.google.com/github/MAHA06/ThesisOulu/blob/master/Adversarials/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# from google.colab import drive
# drive.mount(‘/content/gdrive’)

In [29]:
import math
from torch.autograd.gradcheck import zero_gradients

import seaborn as sns
from torch.autograd import Variable
import torch as torch
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
from sklearn.metrics import plot_confusion_matrix

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms,datasets
np.random.seed(42) 
torch.manual_seed(42)
from sklearn.metrics import confusion_matrix
import pickle
import matplotlib.pyplot as plt
from torchvision.utils import save_image


In [30]:
online=False

img_size=(28,28)
num_classes=10
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.0,), (1.0,))])
dataset = datasets.MNIST(root = './data', train=True, transform = transform, download=True)
train_set, val_set = torch.utils.data.random_split(dataset, [50000, 10000])
test_set = datasets.MNIST(root = './data', train=False, transform = transform, download=True)
train_loader = torch.utils.data.DataLoader(train_set,batch_size=1,shuffle=True) 
val_loader = torch.utils.data.DataLoader(val_set,batch_size=1,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=1,shuffle=True)

In [31]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.relu1=nn.ReLU()
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.relu2=nn.ReLU()
    self.max_p1=nn.MaxPool2d(2)
    self.dropout1 = nn.Dropout2d(0.25)
    self.fc1 = nn.Linear(9216, 128)
    self.relu3=nn.ReLU()
    self.dropout2 = nn.Dropout2d(0.5)

    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = self.relu1(x)
    x = self.conv2(x)
    x = self.relu2(x)
    x = self.max_p1(x)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = self.relu3(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output




In [32]:
use_cuda=False
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
print("Using the device:",device)

Using the device: cpu


In [33]:
def fitModel(model,optimizer,scheduler,criterion,device,train_loader,val_loader,epochs):
  data_loader={'train':train_loader,'val':val_loader}
  print("Starting to fit the model")
  train_error,test_error=[],[]
  best_points=np.zeros(img_size+(num_classes,),dtype=float)
  best_scores=np.full(num_classes,-np.inf)
  print(best_scores[1])
  for epc in range(epochs):
    correct=0
    total=0
    loss_epoch,val_loss_epoch=0,0
    for phase in ('train','val'):

      for i,data in enumerate(data_loader[phase]):
        
        input,label=data[0].to(device),data[1].to(device)
        # print(input.shape)
        out=model(input)
        # print(out)
        loss=criterion(out,label)
        predict_label=torch.argmax(out,dim=1)
        out=out.squeeze().detach().cpu().numpy()
        if phase == 'train':
          optimizer.zero_grad()

          loss.backward()
          optimizer.step()
          loss_epoch+=loss.item()
          # print(predict_label.item(),label.item())
          # print(out[predict_label.item()],best_scores[predict_label.item()])
          # print(out)
          if epc == epochs-1 and predict_label.item()==label.item() and out[predict_label.item()]>best_scores[predict_label.item()]:#if last epoch and the predicted label is correct
            best_scores[predict_label.item()]=out[predict_label.item()]
            best_points[:,:,predict_label]=input.squeeze().detach().cpu().numpy()
            # print(label.item(),out[label.item()])
            # print("The scores were updated:",best_scores)
        else:
          total+=1
          if predict_label==label:
            correct+=1
          val_loss_epoch+=loss.item()
    scheduler.step(val_loss_epoch/len(val_loader))
    print("Epoch : {} Accuracy : {}".format(epc+1,correct/total))
    print("Epoch : {} Train Loss : {} Eval Loss : {}".format(epc+1,loss_epoch/len(train_loader),val_loss_epoch/len(val_loader)))
    train_error.append(loss_epoch)
    test_error.append(val_loss_epoch)
  
  return train_error,test_error,best_points,best_scores
        
          




In [34]:
model=Net().to(device)

optimizer = optim.Adam(model.parameters(),lr=0.0001, betas=(0.9, 0.999))
criterion = nn.NLLLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)


###############


In [35]:
## RUN THE TRAINING
if(online):
  loss,val_loss,best_points,best_scores=fitModel(model,optimizer,scheduler,criterion,device,train_loader,val_loader,10)


In [36]:
if online:
  print(best_points.shape)
  plt.imshow(best_points[:,:,3])
  v=torch.tensor(best_points[:,:,3].reshape(1,1,28,28),dtype=torch.float32)
  print(model(v.to(device)))
  print(torch.argmax(model(v.to(device))))
  print(best_scores)

In [37]:
if online:
  filename = './model_MNIST'
  pickle.dump(model, open(filename, 'wb'))
  filename = './model_MNIST_cpu'
  pickle.dump(model.cpu(), open(filename, 'wb'))
  filename = './scores.mf'
  best_markers={"b_p":best_points,"b_s":best_scores}
  pickle.dump(best_markers,open(filename,'wb'))

In [38]:
def deepfool(image, net, num_classes=10, overshoot=0.02, max_iter=50):

    """
       :param image: Image of size HxWx3
       :param net: network (input: images, output: values of activation **BEFORE** softmax).
       :param num_classes: num_classes (limits the number of classes to test against, by default = 10)
       :param overshoot: used as a termination criterion to prevent vanishing updates (default = 0.02).
       :param max_iter: maximum number of iterations for deepfool (default = 50)
       :return: minimal perturbation that fools the classifier, number of iterations that it required, new estimated_label and perturbed image
    """
    is_cuda = torch.cuda.is_available()

    if is_cuda:
        print("Using GPU")
        image = image.cuda()
        net = net.cuda()
    else:
        print("Using CPU")


    f_image = net.forward(image).data.cpu().numpy().flatten()
    I = (np.array(f_image)).flatten().argsort()[::-1]

    I = I[0:num_classes]
    label = I[0]

    input_shape = image.shape
    pert_image = copy.deepcopy(image)
    w = np.zeros(input_shape)
    r_tot = np.zeros(input_shape)

    loop_i = 0

    x = pert_image
    fs = net.forward(x)
    fs_list = [fs[0,I[k]] for k in range(num_classes)]
    k_i = label

    while k_i == label and loop_i < max_iter:

        pert = np.inf
        fs[0, I[0]].backward(retain_graph=True)
        grad_orig = x.grad.data.cpu().numpy().copy()

        for k in range(1, num_classes):
            zero_gradients(x)

            fs[0, I[k]].backward(retain_graph=True)
            cur_grad = x.grad.data.cpu().numpy().copy()

            # set new w_k and new f_k
            w_k = cur_grad - grad_orig
            f_k = (fs[0, I[k]] - fs[0, I[0]]).data.cpu().numpy()

            pert_k = abs(f_k)/np.linalg.norm(w_k.flatten())

            # determine which w_k to use
            if pert_k < pert:
                pert = pert_k
                w = w_k

        # compute r_i and r_tot
        # Added 1e-4 for numerical stability
        r_i =  (pert+1e-4) * w / np.linalg.norm(w)
        r_tot = np.float32(r_tot + r_i)

        if is_cuda:
            pert_image = image + (1+overshoot)*torch.from_numpy(r_tot).cuda()
        else:
            pert_image = image + (1+overshoot)*torch.from_numpy(r_tot)

        x = Variable(pert_image, requires_grad=True)
        fs = net.forward(x)
        k_i = np.argmax(fs.data.cpu().numpy().flatten())

        loop_i += 1

    r_tot = (1+overshoot)*r_tot

    return r_tot, loop_i, label, k_i, pert_image
def fgsm_attack(input,epsilon,data_grad):
  pert_out = input + epsilon*data_grad.sign()
  pert_out = torch.clamp(pert_out, 0, 1)
  return pert_out

def ifgsm_attack(input,epsilon,data_grad):
  iter = 10
  alpha = epsilon/iter
  pert_out = input
  for i in range(iter-1):
    pert_out = pert_out + alpha*data_grad.sign()
    pert_out = torch.clamp(pert_out, 0, 1)
    if torch.norm((pert_out-input),p=float('inf')) > epsilon:
      break
  return pert_out

def mifgsm_attack(input,epsilon,data_grad):
  iter=10
  decay_factor=1.0
  pert_out = input
  alpha = epsilon/iter
  g=0
  for i in range(iter-1):
    g = decay_factor*g + data_grad/torch.norm(data_grad,p=1)
    pert_out = pert_out + alpha*torch.sign(g)
    pert_out = torch.clamp(pert_out, 0, 1)
    if torch.norm((pert_out-input),p=float('inf')) > epsilon:
      break
  return pert_out

In [39]:
def JacobianMatrix(model,input):
  print("Input size ",input.shape)
  number_of_piels=input.shape[0]*input.shape[1]
  J = torch.zeros ((1, 784, 10))   # loop will fill in Jacobian
  input.requires_grad = True
  preds = model (input)
  for  i in range (10):
      grd = torch.zeros ((1, 10))   # same shape as preds
      grd[0, i] = 1    # column of Jacobian to compute
      preds.backward (gradient = grd, retain_graph = True)
      J[:,:,i] = input.grad   # fill in one column of Jacobian
      input.grad.zero_()   # .backward() accumulates gradients, so reset to zero
def JSM_attack(model, input):
  input.requires_grad=True

In [ ]:
activation_in = {}
activation_out={}
def get_activation_input(name):
    def hook(model, input, output):
        activation_in[name] = input[0]
    return hook
def get_activation_output(name):
    def hook(model, input, output):
        activation_out[name] = output.clone().detach()
    return hook
def nth_derivative(f, wrt, n):

    for i in range(n):

        grads = torch.autograd.grad(f, wrt, create_graph=True)[0]
        f = grads.sum()

    return grads
def get_higher_gradients(model,input,label, rank=3):
  grads=[]
  optimizer.zero_grad()
  input.requires_grad=True
  outs=model(input)
  # pred=torch.zeros(num_classes)
  # pred[label.item()]=1
  l=criterion(outs,label)
  aux=torch.autograd.grad(l,input,retain_graph=True,create_graph=True)[0]
  grads.append(aux.reshape(1,-1))
  for r in range(2,rank+1):
    aux=torch.autograd.grad(aux.sum(),input,create_graph=True)[0]
    grads.append(aux.reshape(1,-1))
  return grads
def check_taylor_prediction(model,input,marker_images,check_label,layer_name,rank):
  # plt.imshow(input.squeeze().detach().cpu().numpy())
  # grads=get_higher_gradients(model,input,true_label)

  # grads=get_higher_gradients(model,input,perturbed_label)
  # layer_grads=get_layer_gradients(model,input,check_label,"conv2")

  # input=input.reshape(-1,1)
  # # marker_image=marker_images[:,:,true_label.item()].reshape(-1,1)
  # # marker_image=torch.tensor(marker_image,dtype=torch.float32)
  # # total=1.+torch.mm(grads[0],(input-marker_image))+torch.mm(grads[1],torch.pow((input-marker_image),2))/2
  # # total=1.+torch.mm(grads[0],(input-marker_image))+torch.mm(grads[1],torch.pow((input-marker_image),2))/2
  # marker_image1=marker_images[:,:,perturbed_label.item()].reshape(-1,1)
  # marker_image1=torch.tensor(marker_image1,dtype=torch.float32)
  # total1=torch.mm(layer_grads[0],(input-marker_image1))+torch.mm(layer_grads[1],torch.pow((input-marker_image1),2))/2
  marker_img=torch.tensor(marker_images[:,:,check_label.item()],dtype=torch.float32).unsqueeze(0).unsqueeze(0)

  # total_marker=compute_marker_image_prediction(model,input,marker_img,check_label,layer_name,rank)
  total_unknown=compute_unknown_image_prediction(model,input,marker_img,check_label,layer_name,rank)

  return total_unknown
def compute_marker_image_prediction(model,unk_image,marker_img,marker_label,l_name,rank):
  grads,input_marker,loss_marker=get_layer_gradients(model,marker_img,marker_label,l_name,rank)
  optimizer.zero_grad()
  input_marker=input_marker.view(-1,1)
  # print(input.shape)
  outs=model(unk_image)
  input_layer=activation_in[l_name].view(-1,1)
  total=loss_marker.item()
  for i in range(0,rank):
    total+=torch.mm(grads[i],torch.pow((input_layer-input_marker),i+1))/(math.factorial(i+1))
  return total.item()
def compute_unknown_image_prediction(model,unk_image,marker_image,marker_label,l_name,rank):

  grads,unk_input,loss_unk=get_layer_gradients(model,unk_image,marker_label,l_name,rank)
  optimizer.zero_grad()
  layer_input=unk_input.view(-1,1)
  # print(input.shape)
  outs=model(marker_image)
  input_layer=activation_in[l_name].view(-1,1)
  total=loss_unk.item()
  for i in range(0,rank):
    total+=torch.mm(grads[i],torch.pow((input_layer-layer_input),i+1))/(math.factorial(i+1))
  # total=loss_unk.item()+torch.mm(grads[0],(input_layer-layer_input))+torch.mm(grads[1],torch.pow((input_layer-layer_input),2))/2

  return total.item()

def get_layer_gradients(model,target_img,label,l_name, rank):
  grads=[]
  optimizer.zero_grad()
  # target_img.requires_grad=True
  outs=model(target_img)
  # pred=torch.zeros((1,num_classes),dtype=np.int)
  # pred[label.item()]=1
  loss_value=criterion(outs,label)
  layer_input_data=activation_in[l_name]
  # layer_input_data.requires_grad=True
  aux=torch.autograd.grad(loss_value,layer_input_data,retain_graph=True,create_graph=True)[0]
  grads.append(aux.reshape(1,-1))
  for r in range(1,rank):
    aux=torch.autograd.grad(aux.sum(),layer_input_data,create_graph=True)[0]
    grads.append(aux.reshape(1,-1))
  return grads,layer_input_data,loss_value

def get_marker_images(model,data_loader):
  best_points=np.zeros(img_size+(num_classes,),dtype=float)
  best_scores=np.full(num_classes,np.inf)
  for i,data in enumerate(data_loader):

      input,label=data[0].to(device),data[1].to(device)
      # print(input.shape)
      out=model(input)
      # print(out)
      loss=criterion(out,label)
      predict_label=torch.argmax(out,dim=1)
      out=out.squeeze().detach().cpu().numpy()

      if   predict_label.item()==label.item() and loss<best_scores[predict_label.item()]:#if last epoch and the predicted label is correct
        best_scores[predict_label.item()]=loss
        best_points[:,:,predict_label.item()]=input.squeeze().detach().cpu().numpy()
  best_markers={"b_p":best_points,"b_s":best_scores}
  return best_markers

filename = './model_MNIST_cpu'
model= pickle.load(open(filename, 'rb'))
filename = './scores.mf'
# best_fits=get_marker_images(model,train_loader)
# pickle.dump(best_fits, open(filename, 'wb'))


best_fits=pickle.load(open(filename,'rb'))

def compute_statistics(confusion_matrix):
  FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
  FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
  TP = np.diag(confusion_matrix)
  TN = confusion_matrix.sum() - (FP + FN + TP)

  # Sensitivity, hit rate, recall, or true positive rate
  TPR = TP/(TP+FN)
  print("Sensitivity",TPR)
  # Specificity or true negative rate
  TNR = TN/(TN+FP)
  print("Specificity",TNR)
  # Precision or positive predictive value
  PPV = TP/(TP+FP)
  print("Precision",PPV)
  # Negative predictive value
  NPV = TN/(TN+FN)
  # Fall out or false positive rate
  FPR = FP/(FP+TN)
  # False negative rate
  FNR = FN/(TP+FN)
  # False discovery rate
  FDR = FP/(TP+FP)
  # Overall accuracy
  ACC = (TP+TN)/(TP+FP+FN+TN)
  print("ACC:",ACC)
def testModel(model,criterion,device,test_loader,attack,rank,l_name="conv1"):
  print()
  print("Started testing with A:{} on L:{} with Rank:{}".format(attack,l_name,rank))
  right=0
  totals=0
  fooled=0
  binary=0
  loss_e=0.02
  loser_count=0
  c_matrix_data=[]
  for data,labels in test_loader:
    data,labels=data.to(device),labels.to(device)
    data.requires_grad=True
    outs=model(data)
    true_preds=torch.argmax(outs,dim=1)
    if true_preds!=labels:
      continue
    epsilon=0.1
    true_loss=criterion(outs,labels)
    true_loss.backward()
    data_grad=data.grad.data
    if attack == "fgsm":
      perturbed_data = fgsm_attack(data,epsilon,data_grad)
    elif attack == "ifgsm":
      perturbed_data = ifgsm_attack(data,epsilon,data_grad)
    elif attack == "mifgsm":
      perturbed_data = mifgsm_attack(data,epsilon,data_grad)
    elif attack=="deepfool":
      perturbed_data=deepfool(data,model)
    elif attack == "jacobian":
      print("Implement the jacobian")
    p_out=model(perturbed_data)
    perturb_label=torch.argmax(p_out,dim=1)
    # print(perturb_label.item())

    if perturb_label!=labels:
      totals+=1
      # print("Trying to determine for the true class:",labels.item()," which was labeled as:",perturb_label.item())
      taylors=range(0,num_classes)
      taylors=[check_taylor_prediction(model,perturbed_data,best_fits['b_p'],torch.tensor(lab).reshape(1),layer_name=l_name,rank=rank) for lab in taylors ]
      # taylors=[check_taylor_prediction(model,data,best_fits['b_p'],torch.tensor(lab).reshape(1),layer_name=l_name) for lab in taylors ]
      taylor_np=np.array(taylors)
      winning_labels=np.where((taylor_np>-5) )
      # if(perturb_label.item() in winning_labels[0]):#check if the predicted label holds 0 loss
      #   binary+=1
      # print("Binary ",binary/totals)
      # print("Right min :",loser_count/totals)

      abs_x=[abs(t) for t in taylors]
      max_l=np.argmax(taylors)
      min_l=np.argmin(abs_x)
      c_matrix_data.append([ labels.item(),max_l])
      if max_l == perturb_label.item():
        fooled+=1
      if max_l == labels.item():
        right+=1
        # save_image(data,"./Outs/original.png")
        # save_image(perturbed_data,"./Outs/perturbed_data.png")
        # save_image(perturbed_data-data,"./Outs/gradient visualization.png")
      if min_l == labels.item():
        loser_count+=1
      # r=check_taylor_prediction(model,data,best_fits['b_p'],perturb_label)

  print("Stats for attack type:"+attack +" on the layer:"+l_name+" with a gradient rank:"+str(rank))
  print("Right max :",right/totals)
  print("Fooled max:",fooled/totals)
  print("Right min :",loser_count/totals)
  print("Binary ",binary/totals)
  confusion_array=np.array(c_matrix_data)
  cm =   confusion_matrix(confusion_array[:,0],confusion_array[:,1])
  compute_statistics(cm)
  f = sns.heatmap(cm, annot=True)
  plt.savefig('./Outs/heatMap-'+attack+'_'+l_name+'.png')
  plt.clf()
  return right/totals
# conv2
model.conv1.register_forward_hook(get_activation_input('conv1'))
model.conv2.register_forward_hook(get_activation_input('conv2'))
model.relu1.register_forward_hook(get_activation_input('relu1'))
model.relu1.register_forward_hook(get_activation_input('relu2'))

model.relu3.register_forward_hook(get_activation_input('relu3'))
model.fc1.register_forward_hook(get_activation_input('fc1'))
model.fc1.register_forward_hook(get_activation_input('fc2'))

print("Starting")
ranks=[1,2,3,4]
layers=["conv1","relu1","conv2","relu2","fc1","relu3","fc2"]
attack_types=["fgsm","mifgsm","ifgsm"]

for att in attack_types:

  for r in ranks:
    acc_mifgsm=[]
    for l in layers:
      t=testModel(model,criterion,'cpu',test_loader,att,r,l_name=l)
      acc_mifgsm.append(t)
    plt.plot(layers,acc_mifgsm)
    plt.savefig('./Outs/{}_R{}.png'.format(att,r))
    plt.clf()



print("Done")

Starting
STarted testing with A:fgsm on L:conv1 with Rank:1

Stats for attack type:fgsm on the layer:conv1 with a gradient rank:1
Right max : 0.4373198847262248
Fooled max: 0.3811239193083574
Right min : 0.3170028818443804
Binary  0.0
Sensitivity [0.43478261 0.234375   0.52803738 0.22340426 0.44230769 0.65605096
 0.38582677 0.44720497 0.44444444 0.3       ]
Specificity [0.94768764 0.98338369 0.94378194 0.93353941 0.94805195 0.89033306
 0.97145123 0.94784026 0.9052901  0.89825119]
Precision [0.3030303  0.40540541 0.63128492 0.19626168 0.51879699 0.43277311
 0.57647059 0.52941176 0.46376812 0.23353293]
ACC: [0.9221902  0.94884726 0.879683   0.88544669 0.89121037 0.86383285
 0.91786744 0.88976945 0.83357349 0.84221902]
STarted testing with A:fgsm on L:relu1 with Rank:1

Stats for attack type:fgsm on the layer:relu1 with a gradient rank:1
Right max : 0.437727603787327
Fooled max: 0.37509104151493083
Right min : 0.30007283321194467
Binary  0.0
Sensitivity [0.38333333 0.14035088 0.53       0